# Introduction to Data Analysis

# What is Data Analysis?

# Pandas Recap

In [1]:
import pandas as pd
df = pd.read_csv("expenses.csv")
print(df.head())

         Date       Category  Amount           Description
0  2025-08-01           Food    15.5         Lunch at cafe
1  2025-08-02      Transport     7.2              Bus fare
2  2025-08-03  Entertainment    25.0          Movie ticket
3  2025-08-04           Food    12.0  Dinner at restaurant
4  2025-08-05      Utilities    45.0      Electricity bill


# Summarizing Data

In [2]:
print(df.describe())
print(df["Amount"].mean())  # average
print(df["Amount"].median())  # middle value
print(df["Amount"].std())  # spread

          Amount
count   5.000000
mean   20.940000
std    14.947174
min     7.200000
25%    12.000000
50%    15.500000
75%    25.000000
max    45.000000
20.94
15.5
14.947173645877003


# Sorting & Filtering

In [3]:
# Filter high expenses
high_expenses = df[df["Amount"] > 20]

# Sort by amount
sorted_df = df.sort_values("Amount", ascending=False)

# Counting Categories

In [4]:
print(df["Category"].value_counts())

Category
Food             2
Transport        1
Entertainment    1
Utilities        1
Name: count, dtype: int64


# Grouping Data

In [5]:
category_totals = df.groupby("Category")["Amount"].sum()
print(category_totals)

Category
Entertainment    25.0
Food             27.5
Transport         7.2
Utilities        45.0
Name: Amount, dtype: float64


# Correlations

In [6]:
print(df.corr(numeric_only=True))

        Amount
Amount     1.0


Shows relationships between numeric columns.

# In-Class Activity